Introduction

Import statements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [ ]:
Load the 2020 data set and the reference files.

In [2]:
a = pd.read_csv("data/2020/2020data.csv")
b = pd.read_csv('data/2020/2020datasimp.csv')
c = pd.read_csv('data/2020/map-keys-2020.csv')
d = pd.read_csv("refs/florahelvetica/florahelveticarcsvnew.csv")

print(a.columns)
print(b.columns)
print(c.columns)
print(d.columns)

Index(['id', 'obs', 'place', 'date', 'time', 'sci', 'name', 'notes'], dtype='object')
Index(['place', 'date', 'time', 'sci', 'name'], dtype='object')
Index(['X', 'Y', 'Name', 'description'], dtype='object')
Index(['acceptedtaxonkey', 'acceptedscientificname', 'numberofoccurrences',
       'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species'],
      dtype='object')


In [3]:
c.rename(columns={'X':'lon', 'Y':'lat', 'Name':'place'}, inplace=True)
b.rename(columns={'sci':'species'}, inplace=True)

In [4]:
a.iloc[:5]

,id,obs,place,date,time,sci,name,notes
0,02777507-6356-5477-92eb-6d7aea343265,NaN,alleestrasse 1,2020-09-02,11:09:04 AM,plantago lanceolata,plantain lancéolé,NaN
1,fd359c20-af7c-5560-b412-56bfbbd154e3,NaN,alleestrasse 1,2020-09-02,11:08:19 AM,centaurea nigra,centaurée noire,could be jaucee
2,469cc29c-f377-5312-ba11-f222ca888cec,NaN,alleestrasse 1,2020-09-02,11:06:59 AM,plantago media,plantain moyen,NaN
3,c7613f5f-b22d-5b51-832f-06cb61bc4128,NaN,alleestrasse 1,2020-09-02,11:06:05 AM,chenopodium album agg.,chénopode blanc,NaN
4,9bba2690-f7a5-54b4-bd74-74fd056ccb3a,NaN,alleestrasse 1,2020-09-02,11:05:18 AM,centaurea jacea agg.,centaurée jacée,NaN


In [5]:
b.iloc[:5]

,place,date,time,species,name
0,alleestrasse 1,2020-09-02,11:09:04 AM,plantago lanceolata,plantain lancéolé
1,alleestrasse 1,2020-09-02,11:08:19 AM,centaurea nigra,centaurée noire
2,alleestrasse 1,2020-09-02,11:06:59 AM,plantago media,plantain moyen
3,alleestrasse 1,2020-09-02,11:06:05 AM,chenopodium album agg.,chénopode blanc
4,alleestrasse 1,2020-09-02,11:05:18 AM,centaurea jacea agg.,centaurée jacée


In [6]:
c.iloc[:5]

,lon,lat,place,description
0,7.253285,47.138823,Jura 2,NaN
1,7.791570,46.685610,Sundgraben Port,NaN
2,7.252867,47.137585,Salome 3,NaN
3,7.252760,47.137273,Salome 4,NaN
4,7.252618,47.136619,Alleestrasse 4\n,NaN


In [7]:
d.iloc[:5]

,acceptedtaxonkey,acceptedscientificname,numberofoccurrences,kingdom,phylum,class,order,family,genus,species
0,2701311,juncus effusus l.,8778,plantae,tracheophyta,liliopsida,poales,juncaceae,juncus,juncus effusus
1,2713300,isolepis setacea (l.) r.br.,362,plantae,tracheophyta,liliopsida,poales,cyperaceae,isolepis,isolepis setacea
2,2874621,citrullus lanatus (thunb.) matsumura & nakai,11,plantae,tracheophyta,magnoliopsida,cucurbitales,cucurbitaceae,citrullus,citrullus lanatus
3,8259811,cephalaria alpina schrad.,353,plantae,tracheophyta,magnoliopsida,dipsacales,caprifoliaceae,cephalaria,cephalaria alpina
4,2889934,tropaeolum majus l.,64,plantae,tracheophyta,magnoliopsida,brassicales,tropaeolaceae,tropaeolum,tropaeolum majus


In [8]:
b.columns

Index(['place', 'date', 'time', 'species', 'name'], dtype='object')

In [9]:
b.species

0         plantago lanceolata
1             centaurea nigra
2              plantago media
3      chenopodium album agg.
4        centaurea jacea agg.
                ...          
932          allium carinatum
933          viburnum lantana
934       filipendula ulmaria
935         lythrum salicaria
936       lysimachia vulgaris
Name: species, Length: 937, dtype: object

In [10]:
def remove_agg(x):
    try:
        data=x.remove('agg.')
    except:
        data=x
    return data
 
def join_that(x):
    try:
        data='-'.join([a_name for a_name in x])
    except:
        data=x
    return data

In [1]:
b['speciesx'] = b.species.apply(lambda x: x.split(" "))
b['speciesy'] = b.speciesx.apply(lambda x: remove_agg(x))

#dictionary of species names to replace in b (badname in b : goodname in d)
replace = {'verbanum bonariensis':'verbena bonariensis', 'medicago varia':'medicago sativa', "oenothera":"oenothera biennis", "geranium pratens":"geranium pratense", "senecio jacobaea": "jacobaea vulgaris", "oenothera biennis ": "oenothera biennis", "solidalgo canadensis": "solidago canadensis", "verbascum lynchitis":"verbascum lychnitis", "verbascum negris":"verbascum nigrum","securigea varia": "securigera varia", "melilotus officianalis": "melilotus officinalis", "knautia maxima": "knautia dipsacifolia", "hieracium aurantiacum":"pilosella aurantiaca", "sysimbrium officinale":"sisymbrium officinale", "geranium robertanium":"geranium robertianum", "mycelis muralis": "lactuca muralis", "calamintha nepeta": "clinopodium nepeta", "polygonum persicaria":"persicaria maculosa", "sorbus aria":"aria edulis"}


NameError: name 'b' is not defined

In [12]:
b['speciesy'] = b.speciesy.apply(lambda x:   join_that(x))

In [13]:
b['speciesy']

0      plantago-lanceolata
1          centaurea-nigra
2           plantago-media
3                     None
4                     None
              ...         
932       allium-carinatum
933       viburnum-lantana
934    filipendula-ulmaria
935      lythrum-salicaria
936    lysimachia-vulgaris
Name: speciesy, Length: 937, dtype: object

In [14]:
# the place variable does not match in the data frames
# df['text'].str.split('::').str[0]
c['place'] = c.place.str.split('\n').str[0]
d['species'] = d.species.str.split('\n').str[0]
# get rid of uppercase
c.place = c.place.str.lower()
b.place = b.place.str.lower()
b.species = b.species.str.lower()
d.species = d.species.str.lower()

# get rid of spaces
c.place = c.place.map(lambda x:  x.replace(" ","-"))
b.place = b.place.map(lambda x:  x.replace(" ","-"))

In [15]:
# find place names that don't match:
a_locs = a.place.unique()
b_locs = b.place.unique()
c_locs = c.place.unique()
b_species = b.species.unique()
d_species = d.species.unique()

# get the symetric difference between the different place name lists
placenames = list(set(c_locs) | set(b_locs))
intersect_placenames = list(set(c_locs) & set(b_locs))
print(F"There are {len(placenames)} different place names\n")
print(F"\nThere are {len(b_locs)} different names in b\n")
print(F"\nThere are {len(c_locs)} different names in c\n")
print(F"\nThere are {len(intersect_placenames)} shared between b and c")

#find the species in the data set that are not in the flora helvetica set
speciesnames = list(set(d_species) | set(b_species))
intersect_speciesnames = list(set(d_species) & set(b_species))
in_b_not_d = list(set(b_species) - set(intersect_speciesnames))
print(F"There are {len(placenames)} different species names\n")
print(F"\nThere are {len(b_species)} different names in b\n")
print(F"\nThere are {len(d_species)} different names in d\n")
print(F"\nThere are {len(intersect_speciesnames)} shared between b and d")
print(F"\nThere are {len(in_b_not_d)} in b not in d")

There are 111 different place names


There are 111 different names in b


There are 111 different names in c


There are 111 shared between b and c
There are 111 different species names


There are 256 different names in b


There are 3418 different names in d


There are 179 shared between b and d

There are 77 in b not in d


In [16]:
print(F"The shared species names:\n\n{intersect_placenames}")

The shared species names:

['boujean-3', 'pieterlen-stockweg', 'cygnes-lot-4', 'cygnes-lot-5', 'cygnes-lot-1', 'schlosslifeld-3', 'chemin-de-la-course', 'football-4', 'hayek-1', 'swatch-1', 'cff-path-2', 'frinvillier-fabrique-4', 'gottstatt-3', 'lucherz-seestrasse', 'buren-1', 'taubenloch-1', 'alleestrasse-3', 'college', 'lezard-1', 'chemin-des-voies', 'sundgraben-forest-right', 'football-1', 'rue-alfred-aebi', 'salome-2', 'sundgraben-gorge', 'cff-path-1', 'sureaux', 'frinvillier-fabrique-3', 'bluets-3', 'rue-de-leau-3', 'truite-1', 'schlosslifeld-2', 'sonnenfeld-1', 'lezard-2', 'sundgraben-port', 'reichenbach', 'truite-5', 'schussinsel-6', 'gottstatt-2', 'pery-taubenlochweg', 'truite-4', 'cff-lot-2', 'saint-ursanne-falls', 'weissenau', 'buren-3', 'buren-2', 'chapelle', 'swatch-2', 'ligerz-favorite', 'energie-2', 'cheyres-reserve', 'cff-path-3', 'sonnenfeld-2', 'football-2', 'schussinsel-2', 'twann-gorges', 'boujean-1', 'schussinsel-3', 'parc-muni', 'sundgraben-bridge-left', 'cygnes-lo

In [17]:
# places in b that do not match c:
b_no_match = [x for x in b_locs if x not in c_locs]
b_no_match
in_b_not_d

['knautia arvensis agg.',
 'sorbus aria agg.',
 'geranium pratens',
 'ranunculus acris agg.',
 'dipsacus fullonum',
 'oenothera biennis aggr.',
 'senecio jacobaea',
 'malus sylvestris agg.',
 'polygonum persicaria',
 'rubus fruticosus',
 'x',
 'lotus corniculatus agg.',
 'verbanum bonariensis ',
 'erigeron annuus',
 'erodium cicutarium agg.',
 'anemone sylvestris',
 'geranium robertanium',
 'leucanthemum vulgare agg.',
 'rorippa islandica agg.',
 'galeopsis tetrahit agg.',
 'prunus spinosa agg.',
 'stachys recta agg.',
 'knautia maxima',
 'verbascum negris',
 'campanula rotundifolia agg.',
 'verbascum lynchitis',
 'galium mollugo agg.',
 'packera aurea',
 'artemisia vulgaris agg.',
 'hylotelephium telephium agg.',
 'salvia pratensis agg.',
 'onobrychis viciifolia agg.',
 'hieracium morulum',
 'centaurea jacea agg.',
 'dianthus carthusianorum agg.',
 'salix nigra',
 'taraxacum',
 'rubus rubus sect.',
 'securigea varia',
 'solidalgo canadensis',
 'rubus',
 'mycelis muralis',
 'linaria vu

In [18]:
type(b.species)

pandas.core.series.Series

In [19]:
replace = {'badname':'goodname'}
type(replace)

dict

In [20]:
replace['badname']

'goodname'

In [21]:
b['a_new_name'] = b.species.map(lambda x: F"{x}three")

In [22]:
b.a_new_name

0         plantago lanceolatathree
1             centaurea nigrathree
2              plantago mediathree
3      chenopodium album agg.three
4        centaurea jacea agg.three
                  ...             
932          allium carinatumthree
933          viburnum lantanathree
934       filipendula ulmariathree
935         lythrum salicariathree
936       lysimachia vulgaristhree
Name: a_new_name, Length: 937, dtype: object

In [23]:
#attempt to correct species names

#function fixes the scientific name in the 4th position of the 
y = b.species
for k in range(len(y[3])):
    if y[3][k] == "verbanum bonariensis":
        y[3][k] = "verbena bonariensis"
    if y[3][k] == "medicago varia":
        y[3][k] = "medicago sativa"
    if y[3][k] == "geranium pratens":
        y[3][k] = "geranium pratense"
    if y[3][k] == "oenothera":
        y[3][k] = "oenothera biennis"
    if y[3][k] == "senecio jacobaea":
        y[3][k] = "jacobaea vulgaris"
    if y[3][k] == "oenothera biennis ":
        y[3][k] = "oenothera biennis"
    if y[3][k] == "solidalgo canadensis":
        y[3][k] = "solidago canadensis"
    if y[3][k] == "verbascum lynchitis":
        y[3][k] = "verbascum lychnitis"
    if y[3][k] == "verbascum negris":
        y[3][k] = "verbascum nigrum"
    if y[3][k] == "securigea varia":
        y[3][k] = "securigera varia"
    if y[3][k] == "melilotus officianalis":
        y[3][k] = "melilotus officinalis"
    if y[3][k] == "knautia maxima":
        y[3][k] = "knautia dipsacifolia"
    if y[3][k] == "hieracium aurantiacum":
        y[3][k] = "pilosella aurantiaca"
    if y[3][k] == "sysimbrium officinale":
        y[3][k] = "sisymbrium officinale"
    if y[3][k] == "geranium robertanium":
        y[3][k] = "geranium robertianum"
    if y[3][k] == "mycelis muralis":
        y[3][k] = "lactuca muralis"
    if y[3][k] == "calamintha nepeta":
        y[3][k] = "clinopodium nepeta"
    if y[3][k] == "polygonum persicaria":
        y[3][k] = "persicaria maculosa"
    if y[3][k] == "sorbus aria":
        y[3][k] = "aria edulis"
b.species = y
new_b_species = b.species.unique()
new_d_species = d.species.unique()
new_speciesnames = list(set(new_d_species) | set(new_b_species))
new_intersect_speciesnames = list(set(new_d_species) & set(new_b_species))
new_in_b_not_d = list(set(new_b_species) - set(new_intersect_speciesnames))
print(F"There are {len(new_speciesnames)} different species names\n")
print(F"\nThere are {len(new_b_species)} different names in b\n")
print(F"\nThere are {len(new_d_species)} different names in d\n")
print(F"\nThere are {len(new_intersect_speciesnames)} shared between b and d")
print(F"\nThere are {len(new_in_b_not_d)} in b not in d")


There are 3495 different species names


There are 256 different names in b


There are 3418 different names in d


There are 179 shared between b and d

There are 77 in b not in d


Clean 2020 data set and reference files

In [24]:
data2020 = utility.removelistreturns(data2020)
floralist = utility.removelistreturns(floralist)
data2020 = utility.makenestedlist(data2020)
floralist = utility.makenestedlist(floralist)
data2020 = utility.arrange2020data(data2020)
floralist = utility.arrangeflorahelveticadata(floralist)
data2020[3] = utility.removenameendings(data2020[3])
data2020 = utility.fixscinames(data2020)
data2020 = utility.fixdate(data2020)
data2020 = utility.fixtime(data2020)

NameError: name 'utility' is not defined

Initial data analysis

In [ ]:
#total number of different values for each variable in data.
occurencesummary = utility.occurencelists(data2020)
print(occurencesummary[3])
newfile = open("occurencesummary.txt", 'w')
newfile.write(occurencesummary)
newfile.close()
comparison = utility.compared(occurencesummary[3][1], floralist[9])
print(comparison)